In [77]:
%load_ext autoreload
%autoreload 2

import os
import fitz
import json
import yaml
import regex

from utils import  TextWord,create_text_lines, merge_text_lines
from detect_language import detect_language_of_document
from keyword_finding import get_keywords_by_language, find_keywords, TOC

base_dir = os.getcwd()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
pdf_path = "data/NAB/Berichte_NAB 10-025_Kurzarbeitsprogram Geothermiebohrung Schlattingen.PDF"
output_path = "data/predictions.json"

with open(os.path.join(base_dir, "matching_params.yml"), "r") as params_file:
    matching_params = yaml.safe_load(params_file)

In [29]:
def extract_by_bookmarks(doc: fitz.Document) :
        """Extracts TOC using PDF bookmarks."""
        toc = doc.get_toc()
        if toc:
            entries = [{"heading": title, "page": page, "level": level} for level, title, page in toc]
            return TOC(entries=entries)
        return None

def extract_by_keyword(language, matching_params):      
    pass

In [92]:
with fitz.open(pdf_path) as doc:
    text_lines = create_text_lines(doc) # doesnt correctly split lines yet ...
    #merged_text_lines = merge_text_lines(text_lines)
    for line in text_lines:
       # print(line.line_text())

        pass

In [49]:
with fitz.Document(pdf_path) as doc:
    language = detect_language_of_document(doc)

    table_of_content = extract_by_bookmarks(doc)
    keywords = get_keywords_by_language(language, matching_params)
    found_keywords = []
    for page_index, page in enumerate(doc):
        page_number = page_index + 1
        words = []
        words_by_line = {}

        for x0, y0, x1, y1, word, block_no, line_no, _word_no in page.get_text("words"):
            rect = fitz.Rect(x0, y0, x1, y1) * page.rotation_matrix
            text_word = TextWord(rect= rect, text = word, page = page_number)
            words.append(text_word)
            key=find_keywords(text_word, keywords)
            if key:
                found_keywords.append(key)
                
    table_of_content=table_of_content.to_dict()

TextLine(Rect(10.0, 20.0, 100.0, 30.0),Hello World)
Cannot create an empty TextLine.


In [41]:
with open(output_path, "w", encoding = "utf-8") as file:
    json.dump(table_of_content, file)

In [16]:
def show_image(item, title=""):
    DPI = 150  # use this resolution
    import numpy as np
    import matplotlib.pyplot as plt

    # %matplotlib inline
    pix = item.get_pixmap(dpi=DPI)
    img = np.ndarray([pix.h, pix.w, 3], dtype=np.uint8, buffer=pix.samples_mv)
    plt.figure(dpi=DPI)  # set the figure's DPI
    plt.title(title)  # set title of image
    _ = plt.imshow(img, extent=(0, pix.w * 72 / DPI, pix.h * 72 / DPI, 0))

if not hasattr(fitz.Page, "find_tables"):
    raise RuntimeError("This PyMuPDF version does not support the table feature")

In [93]:
with fitz.Document(pdf_path) as doc:
    page = doc[11]
    tabs = page.find_tables()  # detect the tables
    for i,tab in enumerate(tabs):  # iterate over all tables
        for cell in tab.header.cells:
            page.draw_rect(cell,color=fitz.pdfcolor["red"],width=0.3)
        page.draw_rect(tab.bbox,color=fitz.pdfcolor["green"])
        print(f"Table {i} column names: {tab.header.names}, external: {tab.header.external}")
        
    #show_image(page, f"Table & Header BBoxes")